In [ ]:
%%bigquery
select * from retails_raw.supplier
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,s_suppkey,s_nationkey,s_comment,s_name,s_address,s_phone,s_acctbal,Date
0,7,0,carefully even packages affix furiously pendin...,Supplier#000000007,P5YDcLfpLaVkJcPnqeMv,400-485-7616,1680.44,2024-02-02 22:02:46.597115+00:00
1,76,0,fluffily even requests haggle ironic courts. d...,Supplier#000000076,5OWkvA4ddFb11A1dNt7rCkDHzDOC,649-580-3618,6300.01,2024-02-02 22:02:46.597115+00:00
2,95,0,slyly regular packages sleep blithely furiousl...,Supplier#000000095,OYPvyNkCnzXUVvc6xtEY 1,940-494-9817,8879.22,2024-02-02 22:02:46.597115+00:00
3,124,0,slyly final packages kindle slyl,Supplier#000000124,"eqxPcUJix,gZc",905-708-6551,1689.36,2024-02-02 22:02:46.597115+00:00
4,128,0,slyly ironic frays should doubt furiously ex,Supplier#000000128,"XNLpBq5,uVtHUt50jVFfn65pIB",972-531-1952,9782.86,2024-02-02 22:02:46.597115+00:00


In [ ]:
%%bigquery
# the supplier data set has coloumns that represent the supplier along with its identification and its account balance
select distinct  s_suppkey, s_name, s_phone, s_address, s_acctbal
from retails_raw.supplier
order by s_name, s_address, s_acctbal, s_suppkey
limit 5


Query is running:   0%|          |

Downloading:   0%|          |

,s_suppkey,s_name,s_phone,s_address,s_acctbal
0,1,Supplier#000000001,800-807-9579,",wWs4pnykQOFl8mgVCU8EZMXqZs1w",3082.86
1,2,Supplier#000000002,348-617-6055,WkXT6MSAJrp4qWq3W9N,3009.73
2,3,Supplier#000000003,471-986-9888,KjUqa42JEHaRDVQTHV6Yq2h,9159.78
3,4,Supplier#000000004,893-133-4384,dxp8WejdtFKFPKa Q7Emf0RjnKx3gR3,9846.01
4,5,Supplier#000000005,752-877-4449,W9VO4vl4dfoDYZ RhawP8xLoc,-74.94


In [ ]:
%%bigquery
select (select count(*) from retails_raw.supplier) as total_supplier_count,
  (select count(*) from (
    select distinct  s_suppkey, s_name, s_phone, s_address, s_acctbal
    from retails_raw.supplier
    order by s_name, s_address, s_acctbal, s_phone, s_suppkey
    limit 5)) as unique_supplier_count
      #from retails_raw.airlines)) as unique_flight_count


Query is running:   0%|          |

Downloading:   0%|          |

,total_supplier_count,unique_supplier_count
0,10000,5


# INVENTORY

In [ ]:

%%bigquery
create or replace table retails_stg.Inventory as
  select
  Product_ID, Product_Name,
  cast(Beginning_Inventory as INT64) as Beginning_Inventory,
  cast(Inventory_Recieved as INT64) as Inventory_Recieved,
  cast(Inventory_Sold as INT64) as  Inventory_Sold,
  cast(Ending_Inventory as INT64) as Ending_Inventory,
  cast(Predicted_Sales as INT64) as Predicted_Sales,
  'kaggle' as data_source,
  Date as load_time
  from
    (select Product_ID, Product_Name, Beginning_Inventory, Inventory_Recieved, Inventory_Sold, Ending_Inventory, Predicted_Sales, 'kaggel' as data_source, Date
      from retails_raw.inventory)

Query is running:   0%|          |

""


# Primary Keys

In [ ]:
%%bigquery
alter table retails_stg.Inventory
  add primary key (Product_ID) not enforced;

Query is running:   0%|          |

""


# Inventory data set


In [ ]:
%%bigquery
select * from retails_raw.inventory
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,Product_ID,Product_Name,Beginning_Inventory,Inventory_Recieved,Inventory_Sold,Ending_Inventory,Predicted_Sales,Date
0,P001,Laptop,118,19,33,104,28,2024-02-09 02:06:48.794941+00:00
1,P001,Laptop,104,7,27,84,28,2024-02-09 02:06:48.794941+00:00
2,P001,Laptop,84,6,22,68,25,2024-02-09 02:06:48.794941+00:00
3,P001,Laptop,68,10,12,66,10,2024-02-09 02:06:48.794941+00:00
4,P001,Laptop,66,3,9,60,7,2024-02-09 02:06:48.794941+00:00




# **Inventory Action:**
 Noticing the Inventory of a speific product at the beginning vs the End

In [ ]:
%%bigquery
create or replace table retails_stg.InventoryAction as
  select distinct Product_ID,Product_Name, Beginning_Inventory, Ending_Inventory
  from retails_stg.inventory

Query is running:   0%|          |

""


In [ ]:
%%bigquery

SELECT * from retails_stg.InventoryAction
LIMIT 5


Query is running:   0%|          |

Downloading:   0%|          |

,Product_ID,Product_Name,Beginning_Inventory,Ending_Inventory
0,P001,Laptop,118,104
1,P001,Laptop,60,42
2,P001,Laptop,68,66
3,P001,Laptop,84,68
4,P001,Laptop,66,60


# Predict Sales VS Actual Sales

In [ ]:
%%bigquery
CREATE OR REPLACE TABLE retails_stg.Predicted_Vs_Actual AS
select distinct Product_ID, Product_Name, Predicted_Sales, Inventory_Sold
from retails_stg.Inventory

Query is running:   0%|          |

""


In [ ]:
%%bigquery

SELECT * from retails_stg.Predicted_Vs_Actual
LIMIT 5

Query is running:   0%|          |

Downloading:   0%|          |

,Product_ID,Product_Name,Predicted_Sales,Inventory_Sold
0,P001,Laptop,28,27
1,P001,Laptop,23,23
2,P001,Laptop,21,20
3,P001,Laptop,10,12
4,P001,Laptop,25,22


# Add Foreign Key in Inventory Action

In [ ]:
%%bigquery
alter table retails_stg.InventoryAction add foreign key (Product_Name)
  references retails_stg.Inventory (Product_ID) not enforced;

Query is running:   0%|          |

""


## Add Foreign Key in Predicted vs Sales Table

In [ ]:
%%bigquery
alter table retails_stg.Predicted_Vs_Actual add foreign key (Product_Name)
  references retails_stg.Inventory (Product_ID) not enforced;

Query is running:   0%|          |

""


# Clean up

this drops the immediate staging table

In [ ]:
%%bigquery
drop table retails_stg.Inventory